Just seeing if I installed Spark correctly

In [1]:
import pyspark

spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [2]:
spark.range(6).show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
+---+



---

In [3]:
import pandas as pd
import numpy as np

from pydataset import data
from pyspark.sql.functions import regexp_extract, regexp_replace
from pyspark.sql.functions import *
from pyspark.sql.functions import col, expr

# 1.) Create a spark data frame that contains your favorite programming languages.

 - The name of the column should be language
 - View the schema of the dataframe
 - Output the shape of the dataframe
 - Show the first 5 records in the dataframe

In [4]:
pd_df = pd.DataFrame({'language': ['Python','JavaScript','Java','Jupyter','Nim','C++','R','sql']})

In [5]:
#that sure looks like the name of the column is language to me
pd_df

,language
0,Python
1,JavaScript
2,Java
3,Jupyter
4,Nim
5,C++
6,R
7,sql


In [6]:
#turn it into a spark df
df = spark.createDataFrame(pd_df)
df

DataFrame[language: string]

In [7]:
#view schema
df.schema

StructType(List(StructField(language,StringType,true)))

In [8]:
#the shape
print((df.count(), len(df.columns)))

(8, 1)


In [9]:
#the first 5 rows
df.show(5)

+----------+
|  language|
+----------+
|    Python|
|JavaScript|
|      Java|
|   Jupyter|
|       Nim|
+----------+
only showing top 5 rows



# 2.) Load the mpg dataset as a spark dataframe.

### a.) Create 1 column of output that contains a message like the one below:

*The 1999 audi a4 has a 4 cylinder engine.*


for each vehicle

In [10]:
#bring in our mpg dataset as a spark dataframe
mpg = spark.createDataFrame(data("mpg"))

In [11]:
#just checking
mpg.show()

+------------+------------------+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|             model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+------------------+-----+----+---+----------+---+---+---+---+-------+
|        audi|                a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|                a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|                a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|                a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|                a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
|        audi|                a4|  2.8|1999|  6|manual(m5)|  f| 18| 26|  p|compact|
|        audi|                a4|  3.1|2008|  6|  auto(av)|  f| 18| 27|  p|compact|
|        audi|        a4 quattro|  1.8|1999|  4|manual(m5)|  4| 18| 26|  p|compact|
|        audi|        a4 quattro|  1.8|1999|  4|  auto(l5)|  4| 16| 25|  p|c

In [12]:
#create the message column
mpg.select(concat(lit('The '), mpg.year, lit(' '), mpg.manufacturer, lit(' '), mpg.model, lit(' has a '), 
                mpg.cyl, lit(' cylinder engine.')).alias('Message')).show(5)

+--------------------+
|             Message|
+--------------------+
|The 1999 audi a4 ...|
|The 1999 audi a4 ...|
|The 2008 audi a4 ...|
|The 2008 audi a4 ...|
|The 1999 audi a4 ...|
+--------------------+
only showing top 5 rows



### b.) Transform the trans column so that it only contains either manual or auto.

In [13]:
#select trans column
#replace contents in trans that start with special character, then some characters, to a special character being the last
#replace with nothing
#name the column 
#show
mpg.select('trans',regexp_replace('trans', r'\(\w+\)$', '').alias('transmission')).show()

+----------+------------+
|     trans|transmission|
+----------+------------+
|  auto(l5)|        auto|
|manual(m5)|      manual|
|manual(m6)|      manual|
|  auto(av)|        auto|
|  auto(l5)|        auto|
|manual(m5)|      manual|
|  auto(av)|        auto|
|manual(m5)|      manual|
|  auto(l5)|        auto|
|manual(m6)|      manual|
|  auto(s6)|        auto|
|  auto(l5)|        auto|
|manual(m5)|      manual|
|  auto(s6)|        auto|
|manual(m6)|      manual|
|  auto(l5)|        auto|
|  auto(s6)|        auto|
|  auto(s6)|        auto|
|  auto(l4)|        auto|
|  auto(l4)|        auto|
+----------+------------+
only showing top 20 rows



# 3.) Load the tips dataset as a spark dataframe.

In [14]:
#tips data into spark dataframe
tips = spark.createDataFrame(data("tips"))

In [15]:
#quality assurance
tips.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



### 3a.) What percentage of observations are smokers?

In [16]:
#total number of entries
tips.count()

244

In [17]:
#the number of smokers
tips.filter(tips.smoker == 'Yes').count()

93

In [18]:
#smoker percentage will be the number of smokers divided by number of entries
smoke_percentage = ((tips.filter(tips.smoker == 'Yes').count())/(tips.count()))*100

In [19]:
print(f'The percentage of observations that are smokers is {smoke_percentage}%')

The percentage of observations that are smokers is 38.114754098360656%


### 3b.) Create a column that contains the tip percentage

In [20]:
tip_percentage = round(((col("tip") / col("total_bill"))) * 100,2)

tips.select(
    col("tip").alias("tip_amount"),
    tips.total_bill.alias("bill"),
    tip_percentage.alias("tip%"),
).show(5)

+----------+-----+-----+
|tip_amount| bill| tip%|
+----------+-----+-----+
|      1.01|16.99| 5.94|
|      1.66|10.34|16.05|
|       3.5|21.01|16.66|
|      3.31|23.68|13.98|
|      3.61|24.59|14.68|
+----------+-----+-----+
only showing top 5 rows



In [21]:
#just select, put the calculation in, and give it an alias if you wish
tips = tips.select('*', (tips.tip / tips.total_bill).alias('tip_pct'))

In [22]:
#wow!
tips.show(5)

+----------+----+------+------+---+------+----+-------------------+
|total_bill| tip|   sex|smoker|day|  time|size|            tip_pct|
+----------+----+------+------+---+------+----+-------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|0.05944673337257211|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|0.16054158607350097|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|0.16658733936220846|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2| 0.1397804054054054|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|0.14680764538430255|
+----------+----+------+------+---+------+----+-------------------+
only showing top 5 rows



### 3c.) Calculate the average tip percentage for each combination of sex and smoker.

In [23]:
#group by sex, and whether they smoke or not, then calculate the mean tip pct for each groups.
tips.groupby('sex').pivot('smoker').mean('tip_pct').show()

+------+------------------+-------------------+
|   sex|                No|                Yes|
+------+------------------+-------------------+
|Female|0.1569209707691836|0.18215035269941032|
|  Male|0.1606687151291298|0.15277117520248512|
+------+------------------+-------------------+



# 4.) Use the seattle weather dataset referenced in the lesson to answer the questions below.

In [24]:
#import data from vega
from vega_datasets import data

weather = data.seattle_weather().assign(date=lambda df: df.date.astype(str))
weather = spark.createDataFrame(weather)
weather.show(5)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|
+----------+-------------+--------+--------+----+-------+
only showing top 5 rows



### 4a.) Convert the temperatures to fahrenheit.

In [25]:
#quick online search to convert celcius to fahrenheit
def convert_temp(c):
    '''
    converts celcius to fahrenheit
    '''
    f = (c*(9/5)) +32
    return f

In [26]:
#convert min temps and max temps to fahrenheit
weather = weather.select('*', convert_temp(weather.temp_max).alias('new_temp_max'), convert_temp(weather.temp_min).alias('new_temp_min'))

In [27]:
weather.show(5)

+----------+-------------+--------+--------+----+-------+------------------+------------+
|      date|precipitation|temp_max|temp_min|wind|weather|      new_temp_max|new_temp_min|
+----------+-------------+--------+--------+----+-------+------------------+------------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|55.040000000000006|        41.0|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|             51.08|       37.04|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|             53.06|       44.96|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|             53.96|       42.08|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|48.019999999999996|       37.04|
+----------+-------------+--------+--------+----+-------+------------------+------------+
only showing top 5 rows



### 4b.) Which month has the most rain, on average?

In [28]:
#we're going to need these
from pyspark.sql.functions import month, year, quarter

In [30]:
(
    weather.withColumn("month", month("date"))
    .groupBy("month")
    .agg(sum("precipitation").alias("total_rainfall"))
    .sort("total_rainfall")
    .show()
)

+-----+------------------+
|month|    total_rainfall|
+-----+------------------+
|    7|              48.2|
|    6|             132.9|
|    8|             163.7|
|    5|             207.5|
|    9|235.49999999999997|
|    4|             375.4|
|    2|             422.0|
|    1|465.99999999999994|
|   10|             503.4|
|    3|             606.2|
|   12| 622.7000000000002|
|   11|             642.5|
+-----+------------------+



November has the most rain on average

### 4c.) Which year was the windiest?

In [32]:
(
    weather.withColumn("year", year("date"))
    .groupBy("year")
    .agg(sum("wind").alias("total_wind"))
    .sort("total_wind")
    .show()
)

+----+------------------+
|year|        total_wind|
+----+------------------+
|2013|1100.8000000000006|
|2015|1153.3000000000002|
|2014|1236.5000000000007|
|2012|            1244.7|
+----+------------------+



2012 had the most wind

### 4d.) What is the most frequent type of weather in January?

In [42]:
weather.show(5)

+----------+-------------+--------+--------+----+-------+------------------+------------+
|      date|precipitation|temp_max|temp_min|wind|weather|      new_temp_max|new_temp_min|
+----------+-------------+--------+--------+----+-------+------------------+------------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|55.040000000000006|        41.0|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|             51.08|       37.04|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|             53.06|       44.96|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|             53.96|       42.08|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|48.019999999999996|       37.04|
+----------+-------------+--------+--------+----+-------+------------------+------------+
only showing top 5 rows



In [44]:
weather = weather.withColumn('month', month('date'))

In [45]:
weather.show(5)

+----------+-------------+--------+--------+----+-------+------------------+------------+-----+
|      date|precipitation|temp_max|temp_min|wind|weather|      new_temp_max|new_temp_min|month|
+----------+-------------+--------+--------+----+-------+------------------+------------+-----+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|55.040000000000006|        41.0|    1|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|             51.08|       37.04|    1|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|             53.06|       44.96|    1|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|             53.96|       42.08|    1|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|48.019999999999996|       37.04|    1|
+----------+-------------+--------+--------+----+-------+------------------+------------+-----+
only showing top 5 rows



In [52]:
weather.crosstab('month', 'weather').show()

+-------------+-------+---+----+----+---+
|month_weather|drizzle|fog|rain|snow|sun|
+-------------+-------+---+----+----+---+
|            5|      1| 25|  16|   0| 82|
|           10|      4| 55|  20|   0| 45|
|            1|     10| 38|  35|   8| 33|
|            6|      2| 14|  19|   0| 85|
|            9|      5| 40|   4|   0| 71|
|            2|      4| 36|  40|   3| 30|
|           12|      2| 54|  23|   5| 40|
|            7|      8| 13|  14|   0| 89|
|            3|      3| 36|  37|   6| 42|
|           11|      3| 50|  25|   0| 42|
|            8|      8| 16|   6|   0| 94|
|            4|      4| 34|  20|   1| 61|
+-------------+-------+---+----+----+---+



Fog is the most common weather type for January

### 4e.) What is the average high and low temperature on sunny days in July in 2013 and 2014?

In [56]:
weather = weather.withColumn('year', year('date'))

In [57]:
weather.show(5)

+----------+-------------+--------+--------+----+-------+------------------+------------+-----+----+
|      date|precipitation|temp_max|temp_min|wind|weather|      new_temp_max|new_temp_min|month|year|
+----------+-------------+--------+--------+----+-------+------------------+------------+-----+----+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|55.040000000000006|        41.0|    1|2012|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|             51.08|       37.04|    1|2012|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|             53.06|       44.96|    1|2012|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|             53.96|       42.08|    1|2012|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|48.019999999999996|       37.04|    1|2012|
+----------+-------------+--------+--------+----+-------+------------------+------------+-----+----+
only showing top 5 rows



In [64]:
weather.filter(expr(
    '(year == 2013 OR year == 2014) AND month == 07')).groupby('weather').pivot('year').mean('temp_max').show()

+-------+------------------+------------------+
|weather|              2013|              2014|
+-------+------------------+------------------+
|    fog| 22.96666666666667|25.439999999999998|
|   rain|              22.2|              29.4|
|    sun|26.585185185185193|            27.092|
+-------+------------------+------------------+



In [63]:
weather.filter(expr(
    '(year == 2013 OR year == 2014) AND month == 07')).groupby('weather').pivot('year').mean('temp_min').show()

+-------+------------------+------------------+
|weather|              2013|              2014|
+-------+------------------+------------------+
|    fog|13.133333333333335|14.440000000000001|
|   rain|              15.0|              15.0|
|    sun|13.981481481481483|14.400000000000002|
+-------+------------------+------------------+



 - Average low in July 2013 for sunny days is 13.98
 - Average high in July 2013 for sunny days is 26.59
 
 - Average low in July 2014 for sunny days is 14.40
 - Average high in July 2014 for sunny days is 27.09

### 4f.) What percentage of days were rainy in q3 of 2015?

In [65]:
weather = weather.withColumn('quarter', quarter('date'))

In [66]:
weather.show(5)

+----------+-------------+--------+--------+----+-------+------------------+------------+-----+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|      new_temp_max|new_temp_min|month|year|quarter|
+----------+-------------+--------+--------+----+-------+------------------+------------+-----+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|55.040000000000006|        41.0|    1|2012|      1|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|             51.08|       37.04|    1|2012|      1|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|             53.06|       44.96|    1|2012|      1|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|             53.96|       42.08|    1|2012|      1|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|48.019999999999996|       37.04|    1|2012|      1|
+----------+-------------+--------+--------+----+-------+------------------+------------+-----+----+-------+
only showing top 5 

In [67]:
rain_q3_2015 = weather.filter(expr('year == 2015 AND quarter == 3'))

In [68]:
rain_q3_2015.show(5)

+----------+-------------+--------+--------+----+-------+-----------------+-----------------+-----+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|     new_temp_max|     new_temp_min|month|year|quarter|
+----------+-------------+--------+--------+----+-------+-----------------+-----------------+-----+----+-------+
|2015-07-01|          0.0|    32.2|    17.2| 4.3|    sun|89.96000000000001|            62.96|    7|2015|      3|
|2015-07-02|          0.0|    33.9|    17.8| 3.4|    sun|            93.02|64.03999999999999|    7|2015|      3|
|2015-07-03|          0.0|    33.3|    17.8| 2.6|    sun|            91.94|64.03999999999999|    7|2015|      3|
|2015-07-04|          0.0|    33.3|    15.0| 2.9|    sun|            91.94|             59.0|    7|2015|      3|
|2015-07-05|          0.0|    32.8|    16.7| 2.1|    sun|91.03999999999999|            62.06|    7|2015|      3|
+----------+-------------+--------+--------+----+-------+-----------------+-----------------+---

In [73]:
rain_q3_2015.where(rain_q3_2015.weather=='rain').count() / rain_q3_2015.count()

0.021739130434782608

### 4g.) For each year, find what percentage of days it rained (had non-zero precipitation).

In [80]:
weather.crosstab('year','weather').show()

+------------+-------+---+----+----+---+
|year_weather|drizzle|fog|rain|snow|sun|
+------------+-------+---+----+----+---+
|        2012|     31|  5| 191|  21|118|
|        2015|      7|173|   5|   0|180|
|        2014|      0|151|   3|   0|211|
|        2013|     16| 82|  60|   2|205|
+------------+-------+---+----+----+---+



In [ ]:
# our years are 2012, 2013, 2014, 2015

In [84]:
def percent_day_rain(string):
    pct_rain = (weather.where(expr(string)).where(expr('precipitation > 0')).count()) / (weather.where(expr(string)).count())
    return pct_rain

In [86]:
#2012
percent_day_rain('year==2012')

0.48360655737704916

In [87]:
#2013
percent_day_rain('year == 2013')

0.41643835616438357

In [88]:
#2014
percent_day_rain('year==2014')

0.410958904109589

In [89]:
#2015
percent_day_rain('year==2015')

0.39452054794520547